In [1]:
import pandas as pd
import numpy as np
import joblib
import shap
from sklearn.preprocessing import StandardScaler

best_model_svr = joblib.load('Full_best_svr_model.joblib')
scaler = joblib.load('scaler.joblib')

# Load new data (replace with your actual data source)
new_data = pd.read_csv('../../train.csv')

# Preprocess the new data
# Apply any feature engineering steps that were used on the training data
# For example, encoding categorical variables, creating new features, etc.

# Scale the features
new_data_scaled = scaler.transform(new_data)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- date_time
- is_holiday
- weather_description
- weather_type
Feature names seen at fit time, yet now missing:
- hour
- is_holiday_Columbus Day
- is_holiday_Independence Day
- is_holiday_Labor Day
- is_holiday_Martin Luther King Jr Day
- ...


In [ ]:
# Create a SHAP explainer object using the best model
# Note: The SHAP KernelExplainer can be used if the model is not directly supported by SHAP
explainer = shap.Explainer(best_model_svr)

# Generate SHAP values for the scaled new data
shap_values = explainer(new_data_scaled)

In [ ]:
# Visualize the SHAP values for a single prediction
shap.plots.waterfall(shap_values[0])

# For a global understanding, use summary plots
shap.summary_plot(shap_values, new_data_scaled)

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
import shap

In [1]:
# Load the data
df_raw = pd.read_csv('../../train.csv')

best_model_svr = joblib.load('Full_best_svr_model.joblib')
scaler = joblib.load('scaler.joblib')
# Convert 'date_time' to datetime and sort
df_raw['date_time'] = pd.to_datetime(df_raw['date_time'])
df_raw.sort_values('date_time', inplace=True)

# Define non-numeric columns
non_numeric_cols = ['is_holiday', 'weather_type', 'weather_description']

# Group by 'date_time' and aggregate: mean for numeric columns, mode for non-numeric columns
agg_funcs = {col: 'mean' for col in df_raw.columns if col not in non_numeric_cols}
agg_funcs.update({col: lambda x: x.mode()[0] if not x.mode().empty else np.nan for col in non_numeric_cols})

df_aggregated = df_raw.groupby('date_time').agg(agg_funcs)

# One-hot encode categorical features
df = pd.get_dummies(df_aggregated, columns=non_numeric_cols, drop_first=True)

# Add hour from the 'date_time' column
df['hour'] = df.index.hour

# Feature engineering: create lagged and rolling features
target = 'traffic_volume'
for i in range(1, 4):
    df[f'traffic_volume_lag_{i}'] = df[target].shift(i)
df['traffic_volume_rolling_mean'] = df[target].rolling(window=3).mean().shift(1)
df['traffic_volume_rolling_std'] = df[target].rolling(window=3).std().shift(1)

# Remove rows with NaN values resulting from lagged features
df.dropna(inplace=True)

# Drop the target column and 'date_time' before scaling
df = df.drop(columns=[target, 'date_time'])

# Scale the numerical features
df_scaled = scaler.transform(df)

# Convert scaled data back to DataFrame
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- traffic_volume


In [ ]:
# Create a SHAP explainer object using the best model
explainer = shap.Explainer(best_model_svr)

# Generate SHAP values for the preprocessed data
shap_values = explainer(df_scaled)

# Visualize the SHAP values
shap.plots.waterfall(shap_values[0])

# For a global understanding, use summary plots
shap.summary_plot(shap_values, df_scaled)